#### 1. Install Dependencies

In [ ]:
pip install -r ../requirements.txt -q

#### 2. Verify python-dotenv

In [ ]:
import os
import pinecone
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

#### 3. Summarizing Using Basic Prompt

In [ ]:
# good enough for small amount of text
# downside: no. of tokens should be less than max supported by LLM

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# text to summarize
TEXT = '''
Rust is a multi-paradigm, general-purpose programming language that emphasizes performance, type safety, and concurrency. It enforces memory safety, meaning that all references point to valid memory, without requiring the use of automated memory management techniques such as garbage collection. To simultaneously enforce memory safety and prevent data races, its "borrow checker" tracks the object lifetime of all references in a program during compilation. Rust was influenced by ideas from functional programming, including immutability, higher-order functions, and algebraic data types. It is popular for systems programming.[13][14][15]

Software developer Graydon Hoare created Rust as a personal project while working at Mozilla Research in 2006. Mozilla officially sponsored the project in 2009. In the years following the first stable release in May 2015, Rust was adopted by companies including Amazon, Discord, Dropbox, Google (Alphabet), Meta, and Microsoft. In December 2022, it became the first language other than C and assembly to be supported in the development of the Linux kernel.

Rust has been noted for its rapid adoption,[16] and has been studied in programming language theory research.
'''

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
print(f'tokens: {llm.get_num_tokens(TEXT)}')
messages = [
  SystemMessage(content = 'You are an experience copywriter who is great at summarizing texts'),
  HumanMessage(content = f'Please summarize the following text in a few words. \n Text: {TEXT}'),
]
summary = llm(messages)
print(summary.content)

#### 4. Summarizing Using Prompt Template

In [ ]:
# good enough for small amount of text
# but better than above because we parameterized it using prompt template
# downside: no. of tokens should be less than max supported by LLM

from langchain import PromptTemplate
from langchain.chains import LLMChain

# query template
template = '''
- Please summarize the following text in a few words:
- Text: {text}
- Provide the summary in {lang} language
'''

# prompt template
prompt = PromptTemplate(
  input_variables = ['text', 'lang'],
  template = template,
)

# text to summarize
TEXT = '''
Rust is a multi-paradigm, general-purpose programming language that emphasizes performance, type safety, and concurrency. It enforces memory safety, meaning that all references point to valid memory, without requiring the use of automated memory management techniques such as garbage collection. To simultaneously enforce memory safety and prevent data races, its "borrow checker" tracks the object lifetime of all references in a program during compilation. Rust was influenced by ideas from functional programming, including immutability, higher-order functions, and algebraic data types. It is popular for systems programming.[13][14][15]

Software developer Graydon Hoare created Rust as a personal project while working at Mozilla Research in 2006. Mozilla officially sponsored the project in 2009. In the years following the first stable release in May 2015, Rust was adopted by companies including Amazon, Discord, Dropbox, Google (Alphabet), Meta, and Microsoft. In December 2022, it became the first language other than C and assembly to be supported in the development of the Linux kernel.

Rust has been noted for its rapid adoption,[16] and has been studied in programming language theory research.
'''

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
print(f'tokens: {llm.get_num_tokens(prompt.format(text=TEXT, lang = "English"))}')
chain = LLMChain(prompt = prompt, llm = llm)

summary_english = chain.run({'text': TEXT, 'lang': 'English'})
print(f'\nin english: {summary_english}')

summary_french = chain.run({'text': TEXT, 'lang': 'French'})
print(f'\nin french: {summary_french}')

summary_hindi = chain.run({'text': TEXT, 'lang': 'Hindi'})
print(f'\nin french: {summary_hindi}')

#### 5. Summarizing Using StuffDocumentsChain

In [ ]:
# same as above two but using a StuffDocumentsChain
# pros: makes a single call to LLM
# cons: if the context (doc) is larger than allowed prompt size, we'll get an error

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

# query template
template = '''
- Please summarize the following text in a few words:
- Text: {text}
- Provide the summary in {lang} language
'''

# prompt template
prompt = PromptTemplate(
  input_variables = ['text', 'lang'],
  template = template,
)

# init
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
chain = LLMChain(prompt = prompt, llm = llm)
chain = load_summarize_chain(llm, chain_type='stuff', prompt=prompt, verbose=False)

# obama's speech to summarize
with open('files/obama-inaugural-speech.txt', encoding='utf-8') as f:
  speech = f.read()
docs = [Document(page_content = speech)]
summary = chain.run({'input_documents': docs, 'lang': 'English'})
print(f'\nObama speech summary in english: {summary}')

# trump's speech to summarize
with open('files/trump-inaugural-speech.txt', encoding='utf-8') as f:
  speech = f.read()
docs = [Document(page_content = speech)]
summary = chain.run({'input_documents': docs, 'lang': 'Spanish'})
print(f'\nTrump speech summary in spanish: {summary}')

#### 6. Summarizing Using Map-Reduce For Large Documents

In [ ]:
# allows summarizing very large documents
# with map-reduce, idea is to first create chunks based on token limit
# pass all chunks to LLM
# it then summarizes each chunk
# it then combines the summaries of each chunk to create the final summary
# pros: 
# - allows summarizing very large documents
# - each chunk summary happens in parallel
# cons: 
# - multiple calls to LLM
# - may lose context when summarizing individual chunks
# - prompts are not customizable

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# obama's speech to summarize
with open('files/obama-inaugural-speech.txt', encoding='utf-8') as f:
  speech = f.read()
print(f'tokens: {llm.get_num_tokens(speech)}') # 2787

# using higher chunnk size to reduce number of OpenAI calls
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=50)
chunks = text_splitter.create_documents([speech])
print(f'chunks: {len(chunks)}') # 3

chain = load_summarize_chain(llm, chain_type='map_reduce', verbose=False)
summary = chain.run(chunks)
print(f'\nSummary: {summary}')

# internally LLM used this as 1st prompt to create summary for each chunk
print(chain.llm_chain.prompt.template)
# Output: "Write a concise summary of the following: \n\n\n "{text}" \n\n\n CONCISE SUMMARY:"

# it then used this prompt to combine summaries
print(chain.combine_document_chain.llm_chain.prompt.template)
# Output: "Write a concise summary of the following: \n\n\n "{text}" \n\n\n CONCISE SUMMARY:"

#### 7. Summarizing Using Map-Reduce With Custom Prompts

In [ ]:
# same as above but using prompt templates
# this lets us customize the summary response

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# template for "map" phase
map_prompt = '''
Write a concise summary of the following text. Text: {text}
'''
map_prompt_template = PromptTemplate(
  input_variables = ['text'],
  template = map_prompt,
)

# template for "combine" phase
combine_prompt = '''
Write a concise summary of the following text in {lang} language.
Add a title to the summary.
The summary should have three parts.
Part one should be INTRODUCTION.
Part two should be a list of BULLET POINTS.
Part three should be CONCLUSION.
Text: {text}
CONCISE_SUMMARY:
'''
combine_prompt_template = PromptTemplate(
  input_variables = ['text', 'lang'],
  template = combine_prompt
)

# obama's speech to summarize
with open('files/obama-inaugural-speech.txt', encoding='utf-8') as f:
  speech = f.read()
print(f'tokens: {llm.get_num_tokens(speech)}') # 2787

# using higher chunnk size to reduce number of OpenAI calls
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=50)
chunks = text_splitter.create_documents([speech])
print(f'chunks: {len(chunks)}') # 3

# summarize
chain = load_summarize_chain(
  llm, 
  chain_type='map_reduce', 
  map_prompt=map_prompt_template,
  combine_prompt=combine_prompt_template,
  verbose=False
)
summary = chain.run({'input_documents': docs, 'lang': 'Hindi'})
print(f'\nSummary: {summary}')

#### 8. Summarizing Using Refine For Large Documents

In [ ]:
# same as map-reduce but uses a different algorithm
# instead of summarizing each chunk inidvidually and then combining them
# this algorithm creates summary for 1st chunk and then uses that as context for 2nd chunk
# and so on
# pros:
# - more accurate summaries
# - less lossy than map-reduce
# cons
# - cannot summarize each chunk in parallel since each chunk's summary depends on summary of previous chunk
# - takes much longer to summarize

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# obama's speech to summarize
with open('files/obama-inaugural-speech.txt', encoding='utf-8') as f:
  speech = f.read()
print(f'tokens: {llm.get_num_tokens(speech)}') # 2787

# using higher chunnk size to reduce number of OpenAI calls
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=50)
chunks = text_splitter.create_documents([speech])
print(f'chunks: {len(chunks)}') # 3

chain = load_summarize_chain(llm, chain_type='refine', verbose=False)
summary = chain.run(chunks)
print(f'\nSummary: {summary}')

#### 9. Summarizing Using Refine With Custom Prompts

In [ ]:
# same as above but uses custom prompts
# "existing_answer" in the prompt below is an inbuilt langchain variable
# changing this to something else won't work for refine template

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# template for "first" phase
# remember, there is no summary to combine in this phase
first_prompt = '''
Write a concise summary of the following text. Text: {text}
'''
first_prompt_template = PromptTemplate(
  input_variables = ['text'],
  template = first_prompt,
)

# template for "refine" phase
# summary is generated using previous chunk's summary as context plus current chunk
refine_prompt = '''
Write a concise summary of the following text. 
You are provided with summary upto a certain point: {existing_answer}
In your final summary, add a title.
The summary should have three parts.
Part one should be INTRODUCTION.
Part two should be a list of BULLET POINTS.
Part three should be CONCLUSION.
Text: {text}
'''
refine_prompt_template = PromptTemplate(
  input_variables = ['existing_answer', 'text'],
  template = refine_prompt
)

# obama's speech to summarize
with open('files/obama-inaugural-speech.txt', encoding='utf-8') as f:
  speech = f.read()
print(f'tokens: {llm.get_num_tokens(speech)}') # 2787

# using higher chunnk size to reduce number of OpenAI calls
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=50)
chunks = text_splitter.create_documents([speech])
print(f'chunks: {len(chunks)}') # 3

# summarize
chain = load_summarize_chain(
  llm, 
  chain_type='refine', 
  question_prompt=first_prompt_template,
  refine_prompt=refine_prompt_template,
  return_intermediate_steps=False,
  verbose=False
)
summary = chain.run(chunks)
print(f'\nSummary: {summary}')

#### 10. Summarizing Using LangChain Agents

In [ ]:
# use an external agent to summarize instead of using LLM's internal capabilities
# in this example, we'll query some wikipedia page & summarize it

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.utilities import WikipediaAPIWrapper

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
wikipedia = WikipediaAPIWrapper()

# configure all external tools our agent will use
tools = [
  Tool(
    name='wikipedia',
    func=wikipedia.run,
    description='Query wikipedia for a page',
  )
]

# initialize agent
agent_executor = initialize_agent(
  tools, 
  llm, 
  agent='zero-shot-react-description', 
  verbose=True
)

summary = agent_executor.run('provide a life summary for "Barack Obama"')
print(f'\nSummary: {summary}')